Build a regression model.

In [28]:
import pandas as pd
import statsmodels.api as sm
import numpy as np


In [3]:
# Read the data from the CSV file
poi_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/statistical_modeling/data/poi_api_data.csv')
poi_data = poi_data.drop("Unnamed: 0", axis=1)
poi_data.head()

,business_id,name,category_id,category,address,latitude,longitude,rating,free bikes
0,57f53ed7498e23279caaed8e,Txotx,13003,Bar,"Øvingshotellet, Korsgata, Fredensborg, Grünerl...",59.919337,10.759947,9.1,NaN
1,4d6803f3709bb60c63f9b014,Le Benjamin Bar & Bistro,13148,French Restaurant,"Recharge Skøyen, Oslo kommune, Drammensveien, ...",59.918750,10.758048,9.4,NaN
2,547a5140498e356dc652cfb7,Territoriet,13025,Wine Bar,"85, Økernveien, Marienfryd, Grünerløkka, Oslo,...",59.918887,10.757560,8.8,NaN
3,4bd1eaae77b29c74bb8f8d82,Glød,13003,Bar,"Kirkeveien, Lindern, St. Hanshaugen, Oslo, 045...",59.920272,10.759572,8.2,NaN
4,5547ba21498e3c063aa9a7ba,Bd57,13006,Beer Bar,"38, Uelands gate, Lovisenberg, St. Hanshaugen,...",59.919910,10.757172,8.4,NaN


In [20]:
poi_data_complete = poi_data[poi_data['business_id'].notna()]

X = poi_data_complete[['latitude', 'longitude', 'category_id']]
y = poi_data_complete['rating']

# Fit the logistic regression model
model = sm.OLS(y, sm.add_constant(X)).fit()

# Print the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     182.8
Date:                Sun, 02 Jul 2023   Prob (F-statistic):          3.92e-115
Time:                        23:23:23   Log-Likelihood:                -11210.
No. Observations:                9181   AIC:                         2.243e+04
Df Residuals:                    9177   BIC:                         2.246e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1101.6112     49.623     22.200      

In [32]:
#ratings threshold
threshold = poi_data_complete['rating'].mean()

poi_data_complete['above_threshold'] = np.where(poi_data_complete['rating'] > threshold, 1, 0)

poi_data_complete.head()

/var/folders/59/vc05kjdn7s14r9w654dq4xlh0000gn/T/ipykernel_30108/3166520659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_data_complete['above_threshold'] = np.where(poi_data_complete['rating'] > threshold, 1, 0)


,business_id,name,category_id,category,address,latitude,longitude,rating,free bikes,binned_category,above_threshold
0,57f53ed7498e23279caaed8e,Txotx,13003,Bar,"Øvingshotellet, Korsgata, Fredensborg, Grünerl...",59.919337,10.759947,9.1,NaN,Other,1
1,4d6803f3709bb60c63f9b014,Le Benjamin Bar & Bistro,13148,French Restaurant,"Recharge Skøyen, Oslo kommune, Drammensveien, ...",59.918750,10.758048,9.4,NaN,Other,1
2,547a5140498e356dc652cfb7,Territoriet,13025,Wine Bar,"85, Økernveien, Marienfryd, Grünerløkka, Oslo,...",59.918887,10.757560,8.8,NaN,Other,1
3,4bd1eaae77b29c74bb8f8d82,Glød,13003,Bar,"Kirkeveien, Lindern, St. Hanshaugen, Oslo, 045...",59.920272,10.759572,8.2,NaN,Other,1
4,5547ba21498e3c063aa9a7ba,Bd57,13006,Beer Bar,"38, Uelands gate, Lovisenberg, St. Hanshaugen,...",59.919910,10.757172,8.4,NaN,Other,1


In [35]:
#create a classification model
X = poi_data_complete[['latitude', 'longitude']]
y = poi_data_complete['above_threshold']

# Fit the logistic regression model
model = sm.GLM(y, sm.add_constant(X), family=sm.families.Binomial()).fit()

# Print the model summary
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        above_threshold   No. Observations:                 9181
Model:                            GLM   Df Residuals:                     9178
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5039.2
Date:                Sun, 02 Jul 2023   Deviance:                       10078.
Time:                        23:36:19   Pearson chi2:                 9.19e+03
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01122
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1478.4835    151.076      9.786      0.0

Provide model output and an interpretation of the results. 

# Stretch

How can you turn the regression model into a classification model?

In [115]:
threshold = poi_data_encoded['rating'].mean()
X = poi_data_encoded[['latitude', 'longitude', 'category_id', 'category_Bike Station', 'category_Cafe', 'category_Other', 'category_Restaurant']]

y = (poi_data_encoded[poi_data_encoded['rating'] > threshold]['rating'])

# Fit the logistic regression model
model = sm.Logit(y, sm.add_constant(X)).fit()

# Print the model summary
print(model.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).